<a href="https://colab.research.google.com/github/Az-Ks/AirQo-Ugandan-Air-Quality-Forecast/blob/master/indianda.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
pip install catboost

     |████████████████████████████████| 64.8MB 122kB/s 


In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

from math import sqrt 
import lightgbm as lgb
from sklearn.metrics import mean_squared_error 
from sklearn.model_selection import KFold, train_test_split


from scipy.sparse import csr_matrix
import gc

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from xgboost import XGBRegressor
from catboost import CatBoostRegressor
from lightgbm import LGBMRegressor
from sklearn.ensemble import GradientBoostingRegressor,RandomForestRegressor,BaggingRegressor,AdaBoostRegressor,ExtraTreesRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import StackingRegressor
#from ngboost import NGBRegressor
from sklearn.metrics import mean_squared_log_error
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import KFold,StratifiedKFold, GroupKFold
import datetime
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
# from fastai.tabular import *
import warnings
warnings.filterwarnings("ignore")
np.random.seed(0)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [0]:
# Memory reduction helper function:
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns: #columns
        col_type = df[col].dtypes
        if col_type in numerics: #numerics
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

In [0]:
from IPython.display import HTML
import pandas as pd
import numpy as np
import base64

# download it (will only work for files < 2MB or so)
def create_download_link(df, title = "Download CSV file", filename = "rf.csv"):  
    csv = df.to_csv(index=False)
    b64 = base64.b64encode(csv.encode())
    payload = b64.decode()
    html = '<a download="{filename}" href="data:text/csv;base64,{payload}" target="_blank">{title}</a>'
    html = html.format(payload=payload,title=title,filename=filename)
    return HTML(html)

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
path =  '/content/drive/My Drive/'

In [0]:
train_df = pd.read_csv(path+'train_df.csv')
test_df = pd.read_csv(path+'test_df.csv')
train_df.shape,test_df.shape

((15539, 3087), (5035, 3086))

In [0]:
temp = ["temp{}".format(i) for i in range(121)]
precip = ["precip{}".format(i) for i in range(121)]
rel_humidity = ["rel_humidity{}".format(i) for i in range(121)]
wind_dir = ["wind_dir{}".format(i) for i in range(121)]
wind_spd = ["wind_spd{}".format(i) for i in range(121)]
atmos_press = ["atmos_press{}".format(i) for i in range(121)]

In [0]:
grp_data11_col = pd.read_csv(path+'grp_data11_col.csv')
grp_data11_col = list(grp_data11_col['0'].values)

grp_data21_col = pd.read_csv(path+'grp_data21_col.csv')
grp_data21_col = list(grp_data21_col['0'].values)

grp_data31_col = pd.read_csv(path+'grp_data31_col.csv')
grp_data31_col = list(grp_data31_col['0'].values)

grp_data41_col = pd.read_csv(path+'grp_data41_col.csv')
grp_data41_col = list(grp_data41_col['0'].values)

grp_data51_col = pd.read_csv(path+'grp_data51_col.csv')
grp_data51_col = list(grp_data51_col['0'].values)

rem_org = pd.read_csv(path+'rem_org.csv')
rem_org = list(rem_org['0'].values)

In [0]:
len(rem_org),len(grp_data11_col),len(grp_data21_col),len(grp_data31_col),len(grp_data41_col),len(grp_data51_col)

(116, 511, 409, 817, 307, 205)

In [0]:
f1 = list(set(temp + precip + rel_humidity + wind_dir + wind_spd + atmos_press + rem_org)) # orig
f2 = list(set(f1 + grp_data11_col)) #day
f3 = list(set(f1 + grp_data21_col)) #typeofday
f4 = list(set(f1 + grp_data31_col)) #type_of_day1
f5 = list(set(f1 + grp_data41_col)) #type_of_day2
f6 = list(set(f1 + grp_data51_col)) #type_of_day3
f7 = list(set(f1 + f3 + f5)) #type_of_day day_type_of_day2
f8 = list(set(f1 + f3 + f6)) #type_of_day day_type_of_day3
f9 = list(set(f1 + f4 + f5)) #type_of_day1 day_type_of_day2
f10 = list(set(f1 + f4 + f6)) #type_of_day1 day_type_of_day3
f11 = list(set(f1 + f5 + f6)) #type_of_day2 day_type_of_day3
f12 = list(set(f1 + f3+ f5 + f6)) #type_of_day type_of_day2 day_type_of_day3
f13 = list(set(f1 + f4+ f5 + f6)) #type_of_day1 type_of_day2 day_type_of_day3
f14 = list(set(f1 +f2+ f4+ f5 + f3 + f6)) #type_of_day1 type_of_day2 day_type_of_day3

In [0]:
train = train_df[f7+['target']]
test = test_df[f7]
train.shape,test.shape

((15539, 1557), (5035, 1556))

In [0]:
train_id = train['ID']
test_id = test['ID']
y = train['target']

train.drop(['ID','target'],axis=1,inplace=True)
test.drop('ID',axis=1,inplace=True)
train.shape,test.shape

((15539, 1555), (5035, 1555))

In [0]:
forecastwindte=pd.read_csv('/content/winddirforecasttest ADD THOSE FEATURES TO TEST.csv')
forecastwindtr=pd.read_csv('/content/winddirforecasttrain ADD THOSE COLUMNS TO TRAIN.csv')

In [0]:
X  = train.copy()
Xtest = test.copy()

In [0]:
for i in forecastwindte.columns :
  X[i]=forecastwindtr[i]
  Xtest[i]=forecastwindte[i]

In [0]:
%%time
from lightgbm import LGBMRegressor
lgb = LGBMRegressor()
lgb.fit(X,y)

imp = pd.DataFrame(lgb.feature_importances_,index=X.columns)
l = list(imp[imp[0]>3].index)
len(l)

CPU times: user 1min 10s, sys: 119 ms, total: 1min 11s
Wall time: 36.8 s


In [0]:
len(l),X.shape

(251, (15539, 1573))

In [0]:
X[l].shape

(15539, 251)

In [0]:
from catboost import CatBoostRegressor
errcb2=[]
y_pred_totcb2=[]
from sklearn.model_selection import KFold,StratifiedKFold, TimeSeriesSplit
from sklearn.metrics import mean_squared_error
fold=KFold(n_splits=20)#15#5#10
i=1
for train_index, test_index in fold.split(X,y):
    X_train, X_test = X[l].values[train_index], X[l].values[test_index]
    y_train, y_test = y.values[train_index], y.values[test_index]
    m2  = CatBoostRegressor(n_estimators=5000,eval_metric='RMSE',learning_rate=0.175, random_seed= 42, use_best_model=True )
    m2.fit(X_train,y_train,eval_set=[(X_train,y_train),(X_test, y_test)], early_stopping_rounds=200,verbose=200)
    preds=m2.predict(X_test)
    print("err: ",np.sqrt(mean_squared_error(y_test,preds)))
    errcb2.append(np.sqrt(mean_squared_error(y_test,preds)))
    p2 = m2.predict(Xtest[l])
    y_pred_totcb2.append(p2)
np.mean(errcb2)

0:	learn: 40.0876723	test: 40.0876723	test1: 40.8671914	best: 40.8671914 (0)	total: 82.9ms	remaining: 6m 54s
200:	learn: 18.7261668	test: 18.7261668	test1: 24.3206390	best: 24.2671957 (195)	total: 15.7s	remaining: 6m 14s
400:	learn: 14.0273746	test: 14.0273746	test1: 23.2844559	best: 23.2426862 (384)	total: 31.2s	remaining: 5m 58s
600:	learn: 11.1481095	test: 11.1481095	test1: 22.9916124	best: 22.9214612 (544)	total: 46.8s	remaining: 5m 42s
800:	learn: 9.1067324	test: 9.1067324	test1: 22.9154239	best: 22.8916184 (796)	total: 1m 2s	remaining: 5m 26s
1000:	learn: 7.5375543	test: 7.5375543	test1: 22.8136649	best: 22.8033906 (983)	total: 1m 17s	remaining: 5m 10s
1200:	learn: 6.2964674	test: 6.2964674	test1: 22.7658360	best: 22.7564960 (1192)	total: 1m 33s	remaining: 4m 54s
1400:	learn: 5.3083733	test: 5.3083733	test1: 22.7173545	best: 22.6998592 (1353)	total: 1m 49s	remaining: 4m 41s
1600:	learn: 4.4913219	test: 4.4913219	test1: 22.6589477	best: 22.6589477 (1600)	total: 2m 5s	remaining: 4m

22.790050240627117

In [0]:
np.mean(errcb2)

22.790050240627117

In [0]:
d = {'ID': test_id, 'target': np.mean(y_pred_totcb2, 0)}
sub = pd.DataFrame(data=d)
sub = sub[['ID', 'target']]

In [0]:
sub.to_csv('indianda.csv',index=False)
from google.colab import files
files.download("indianda.csv")